<a href="https://colab.research.google.com/github/Aakarsh204/Pneumonia-Detection-CNN/blob/main/Pneumonia_Detection_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pneumonia Kaggle Classification

##Unzipping




* zip file must be present in drive/MyDrive/< .zip file name >
* zip file data extracted to drive/MyDrive/ML Data Sets/< folder name >
* Checkpointing of models saved to drive/MyDrive/ML Models/< model name >
* Train and Test history of models saved to drive/MyDrive/ML Models/Train and Test Data/< model name >

In [ ]:
from zipfile import ZipFile

# Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/chestxray.zip'

with ZipFile(zip_path, 'r') as zipObj:
    zipObj.extractall('drive/MyDrive/ML Data Sets')

##Hyper-Parameters

Setting the following hyper-parameters:
* Learning rate
* Epochs
* Batch Size
* Patience
* Resolution

In [1]:
# .zip file name
ZIP_NAME = "chestxray.zip"
FOLDER_NAME = "Kaggle Chest X-Ray"

# Loading a Model and its Train and Test History
load_status = True
LOAD_NAME = "CNN_v1"

# Saving a Model (state_dict only)
save_status = True
SAVE_NAME = "CNN_v1"

# Evaluate Model
evaluate_status = True
EVALUATE_NAME = SAVE_NAME

# Learning Rate
LR = 0.001

# Batch Size
BATCH_SIZE = 64

# Number of Epochs
EPOCHS = 10

# Early Stopping Patience ( Stop training if no improvement in test loss after specificed number of epochs )
PATIENCE = 10
early_stop_status = True   # Set to True to turn on Early Stopping

# Input Resolution Image will be resized and passed to CNN
RESIZED_RESOLUTION = 256

## Setup and Prerequisites


###Libraries

In [ ]:
import os
import numpy as np

import torch
from torch import nn, optim

# Import torchvision
import torchvision
from torchvision import datasets

# Data Transforms
from torchvision import transforms as T

# Dataset and Models
from torchvision import datasets, models

# Data Loader
from torch.utils.data import DataLoader

# Mathplotlib
import matplotlib.pyplot as plt

# Pathlib
from pathlib import Path

# Import tqdm for progress bar
from tqdm.auto import tqdm

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Device: {device}")

Device: cuda


##Class Counts

In [ ]:
# Setting up Paths
data_path = Path("data/")
drive_path = Path("drive/MyDrive")
datasets_path = drive_path / "ML Data Sets"
pneumonia_path = datasets_path / FOLDER_NAME

def print_class_counts(dir_path, print_dir = False):

    # Printing directories
    for dirpath, dirnames, filenames in os.walk(dir_path):
        if print_dir:
            print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
def dataTransforms(phase = None):

    if phase == 'train':

        data = T.Compose([
            T.Resize(size=(256,256)),
            T.CenterCrop(size=224),
            T.RandomHorizontalFlip(p=0.5),
            T.RandomRotation(degrees = (-20, +20)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),
        ])

    elif phase == 'test' or phase == 'val':

        data = T.Compose([

                T.Resize(size = (224,224)),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])

    return data

In [ ]:
dataDir = "/content/drive/MyDrive/pneumonia/chest/chest_xray/chest_xray"
test = 'test'
train = 'train'

trainSet = datasets.ImageFolder(os.path.join(dataDir, train),transform = dataTransforms(train))
testSet = datasets.ImageFolder(os.path.join(dataDir, test),transform = dataTransforms(test))

In [ ]:
classNames = trainSet.classes
print("Class Names:", classNames)
print("Class-to-Index Mapping:", trainSet.class_to_idx)

Class Names: ['NORMAL', 'PNEUMONIA']
Class-to-Index Mapping: {'NORMAL': 0, 'PNEUMONIA': 1}


In [ ]:
class_counts = np.array([1341, 3875])  # Example class frequencies
total_samples = np.sum(class_counts)

# Calculate class weights
class_weights = total_samples / (len(class_counts) * class_counts)
print(f"Class weights: {class_weights}")

# Convert to a tensor
class_weights = torch.tensor(class_weights, dtype=torch.float)
if torch.cuda.is_available():
    class_weights = class_weights.cuda()

Class weights: [1.9448173  0.67303226]


In [ ]:
trainLoader = DataLoader(trainSet,batch_size = 64,shuffle = True)
testLoader = DataLoader(testSet,batch_size = 64,shuffle = False)

images, labels = next(iter(trainLoader))
print(images.shape)
print(labels.shape)

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [ ]:
import sys
import torchvision
resnet_model = torchvision.models.resnet50(weights = 'IMAGENET1K_V2')
print(resnet_model)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 134MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        # Add a global average pooling operation to match the expected input of fc layer
        x = torch.mean(x, dim=[2, 3])  # Average over the spatial dimensions (height and width)
        return x

resnet_model.avgpool = Identity()
resnet_model.fc = nn.Linear(in_features=2048, out_features=2)

In [ ]:
# defining the optimizer
optimizer = optim.SGD(resnet_model.parameters(), lr=0.01)

# defining the loss function
criterion = nn.CrossEntropyLoss(weight = class_weights)

# defining the scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
# checking if GPU is available
if torch.cuda.is_available():
    criterion = criterion.cuda()

In [ ]:
if torch.cuda.is_available():
    resnet_model.to("cuda")

Losses = []
for i in range(10):
    running_loss = 0
    for images, labels in trainLoader:

        # Changing images to cuda for GPU
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        # Training pass
        # Sets the gradient to zero
        optimizer.zero_grad()

        output = resnet_model(images)
        loss = criterion(output, labels)

        # This is where the model learns by backpropagating
        # Accumulates the loss for mini batch
        loss.backward()

        # And optimizes its weights here
        optimizer.step()
        Losses.append(loss.item())

        running_loss += loss.item()

    # Scheduler step after each epoch
    scheduler.step()

    print("Epoch {} - Training loss: {}".format(i + 1, running_loss / len(trainLoader)))


Epoch 1 - Training loss: 0.4879041761159897
Epoch 2 - Training loss: 0.26548882573843
Epoch 3 - Training loss: 0.18024961115873378
Epoch 4 - Training loss: 0.1607732931556909
Epoch 5 - Training loss: 0.13397014626990195
Epoch 6 - Training loss: 0.12977995213283144
Epoch 7 - Training loss: 0.11777928524205218
Epoch 8 - Training loss: 0.11754151628069255
Epoch 9 - Training loss: 0.11833742422902066
Epoch 10 - Training loss: 0.11851817418051802


In [ ]:
pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torchmetrics
 # Getting all test predictions
y_preds = []
y_preds_pre_argmax = []
y_labels = []
transfer_model.eval()
with torch.inference_mode():
  for (X, y) in (testLoader):
    X, y = X.to('cuda'), y.to('cuda')
    y_pred = transfer_model(X)
    y_preds_pre_argmax.append(y_pred)
    y_preds.append(y_pred.argmax(dim = 1).cpu())
    y_labels.append(y)
  # Converting list to tensor ( y_preds has shape ([624]) = len(dataLoader) )
y_preds_tensor = torch.cat(y_preds).cpu()
y_preds_pre_argmax_tensor = torch.cat(y_preds_pre_argmax).cpu()
  # Converting test_data lables to tensor ( labels_tensor has shape ([624]) )
labels_tensor = torch.tensor(test_data.targets).cpu()


  # AUROC
auroc_obj = torchmetrics.classification.MulticlassAUROC(num_classes = 2, average = None, thresholds = None)
auroc = auroc_obj(y_preds_pre_argmax_tensor, labels_tensor)
print(f"AUROC for NORMAL (Class 0): {auroc[0]:0.3f}")
print(f"AUROC for PNEUMONIA (Class 1): {auroc[1]:0.3f}")

In [ ]:
from pathlib import Path
# Creating models directory
MODEL_NAME = "transfer_model"
MODEL_PATH = Path("drive/MyDrive/ML Models")
if not MODEL_PATH.is_dir():
  MODEL_PATH.mkdir(parents = True, exist_ok = True)
else:
  print("Model save path (drive/MyDrive/ML Models) already exists.")
# Creating save path
MODEL_NAME = MODEL_NAME + ".pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
# Saving model state_dict
torch.save(obj = transfer_model.state_dict(), f = MODEL_SAVE_PATH)
print(f"Model Saved as {MODEL_NAME}")

Model Saved as transfer_model.pth


In [ ]:
import torch
from sklearn.metrics import roc_auc_score
import numpy as np

# Initialize lists to store true labels and predicted probabilities
all_labels = []
all_probs = []

# Put the model in evaluation mode
resnet_model.eval()

with torch.inference_mode():
    for data in testLoader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')

        outputs = resnet_model(images)

        # Assuming outputs are raw scores (logits)
        probs = torch.softmax(outputs, dim=1)

        # Store the true labels and predicted probabilities
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

# Compute AUROC for each class
auroc_per_class = []
for i in range(all_probs.shape[1]):
    auroc = roc_auc_score(all_labels == i, all_probs[:, i])
    auroc_per_class.append(auroc)

# Compute the average AUROC
average_auroc = np.mean(auroc_per_class)

print(f'AUROC of class NORMAL: {auroc_per_class[0]}')
print(f'AUROC of class PNEUMONIA: {auroc_per_class[1]}')


AUROC of class NORMAL: 0.9347797501643655
AUROC of class PNEUMONIA: 0.9347797501643655
Average AUROC: 0.9347797501643655


In [ ]:
class classify(nn.Module):
    def __init__(self,num_classes=2):
        super(classify,self).__init__()

        self.conv1=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)

        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        self.fc=nn.Linear(in_features=32 * 112 * 112,out_features=num_classes)


    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        output=output.view(-1,32*112*112)
        output=self.fc(output)

        return output

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# defining the optimizer
optimizer = optim.Adam(transfer_model.parameters(), lr=0.01)
# defining the loss function
criterion = nn.CrossEntropyLoss(weight = class_weights)
# checking if GPU is available
if torch.cuda.is_available():
    criterion = criterion.cuda()

In [ ]:
Losses = []
for i in range(10):
    running_loss = 0
    for images, labels in trainLoader:

        #Changing images to cuda for gpu
        if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()

        # Training pass
        # Sets the gradient to zero
        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        #This is where the model learns by backpropagating
        # accumulates the loss for mini batch
        loss.backward()

        #And optimizes its weights here
        optimizer.step()
        Losses.append(loss)

        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(i+1, running_loss/len(trainLoader)))

NameError: name 'model' is not defined

In [ ]:
correct_count, all_count = 0, 0
for images,labels in testLoader:
  for i in range(len(labels)):
    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    img = images[i].view(1, 3, 224, 224)
    with torch.no_grad():
        logps = resnet_model(img)


    ps = torch.exp(logps)
    probab = list(ps.cpu()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 624

Model Accuracy = 0.8830128205128205
